<a href="https://colab.research.google.com/github/Doubles2/All_of_DoubleS2/blob/master/ChuLuo_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

본 저자의 코드에서 hpg는 우리 쪽과 다르기 때문에 제외된 데이터셋을 이용하여 모델 설계를 진행함

참고 : https://github.com/kasuo46/Restaurant_Visitor_Forecasting

import library

In [0]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

Data Load

In [0]:
#구글 드라이브에서 업로드
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
number = '4'
Train = pd.read_csv('/gdrive/My Drive/Colab Notebooks/dataset/2차/train_' + number + '.csv')
#Y_Train = pd.read_csv('/gdrive/My Drive/Colab Notebooks/dataset/2차/train_label_' + number + '.csv')
Test = pd.read_csv('/gdrive/My Drive/Colab Notebooks/dataset/2차/test_' + number + '.csv')
#Y_Test = pd.read_csv('/gdrive/My Drive/Colab Notebooks/dataset/2차/test_label_' + number + '.csv')

train_feature = ['visit_dow', 'visit_month', 'visit_year', 'holiday_flg', 'min_visitors', 'mean_visitors', 'max_visitors', 'count_visitors', 'median_visitors', 'weight']
label_feature = ['visitors']

X_Train = Train[train_feature]
Y_Train = Train[label_feature]
X_Test = Test[train_feature]
Y_Test = Test[label_feature]
del Train, Test

In [0]:
print('X_Train : ', X_Train.shape)
print('Y_Train : ', Y_Train.shape)
print('X_Test : ', X_Test.shape)
print('Y_Test : ', Y_Test.shape)

X_Train :  (380, 10)
Y_Train :  (380, 1)
X_Test :  (95, 10)
Y_Test :  (95, 1)


In [0]:
def min_max_norm(Molecular_Table, denominator_Table):
  import math
  import numpy as np

  alp = math.pow(10, -8)
  mmn_col = Molecular_Table.columns
  for col in mmn_col:
    Molecular_Table[col] = ( Molecular_Table[col] - denominator_Table[col].min() ) / ( denominator_Table[col].max() - denominator_Table[col].min() + alp)
  return Molecular_Table

def z_norm(Molecular_Table, denominator_Table):
  import math
  import numpy as np

  alp = math.pow(10, -8)
  z_col = Molecular_Table.columns
  for col in z_col:
    Molecular_Table[col] = ( Molecular_Table[col] - denominator_Table[col].mean() ) / (denominator_Table[col].std() + alp)
  return Molecular_Table

In [0]:
X_Train['visit_year'] -= 2016
X_Test['visit_year'] -= 2016
X_Test  = min_max_norm(X_Test, X_Train)
X_Train = min_max_norm(X_Train, X_Train)

In [0]:
X_Train.head()

,visit_dow,visit_month,visit_year,holiday_flg,min_visitors,mean_visitors,max_visitors,count_visitors,median_visitors,weight
0,0.666667,0.0,0.0,1.0,0.285714,0.096089,0.000000,1.0,0.178571,0.000000e+00
1,0.833333,0.0,0.0,1.0,0.714286,1.000000,0.680851,1.0,1.000000,3.761558e-12
2,1.000000,0.0,0.0,1.0,1.000000,0.941366,0.744681,0.0,0.857143,2.936442e-11
3,0.000000,0.0,0.0,0.0,0.285714,0.000000,0.212766,1.0,0.000000,1.241314e-10
4,0.166667,0.0,0.0,0.0,0.571429,0.136313,1.000000,1.0,0.035714,3.790680e-10


In [0]:
X_Test.head()

,visit_dow,visit_month,visit_year,holiday_flg,min_visitors,mean_visitors,max_visitors,count_visitors,median_visitors,weight
0,0.333333,0.0,1.0,0.0,0.000000,0.131844,0.361702,1.0,0.071429,1.013123
1,0.500000,0.0,1.0,0.0,0.285714,0.170950,0.276596,1.0,0.142857,1.026384
2,0.666667,0.0,1.0,0.0,0.285714,0.096089,0.000000,1.0,0.178571,1.039783
3,0.833333,0.0,1.0,0.0,0.714286,1.000000,0.680851,1.0,1.000000,1.053321
4,1.000000,0.0,1.0,0.0,1.000000,0.941366,0.744681,0.0,0.857143,1.067001


필요 함수 정의

In [0]:
def rmsle(y, pred):
  from sklearn.metrics import mean_squared_log_error
  return mean_squared_log_error(y, pred)**0.5

GBR


In [0]:
model_gbr = GradientBoostingRegressor(learning_rate=0.2, verbose=True)
model_gbr.fit(X_Train, Y_Train)
pred_train_gbr = np.clip(model_gbr.predict(X_Train), a_min=0.0, a_max=None)
pred_val_gbr = np.clip(model_gbr.predict(X_Test), a_min=0.0, a_max=None)

      Iter       Train Loss   Remaining Time 
         1          81.5617            0.08s
         2          70.2909            0.07s
         3          62.8343            0.07s
         4          57.7282            0.06s
         5          53.3082            0.06s
         6          50.1112            0.06s
         7          47.6156            0.06s
         8          45.0672            0.06s
         9          43.7110            0.06s
        10          41.7993            0.06s
        20          30.7464            0.05s
        30          26.1144            0.04s
        40          22.4692            0.03s
        50          19.5806            0.03s
        60          17.7412            0.02s
        70          15.1128            0.02s
        80          13.4611            0.01s
        90          11.9132            0.01s
       100          10.3999            0.00s


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KNR

In [0]:
model_knr = KNeighborsRegressor(n_neighbors=5, n_jobs=-1)
model_knr.fit(X_Train, Y_Train)
pred_train_knr = np.clip(model_knr.predict(X_Train), a_min=0.0, a_max=None)
pred_val_knr = np.clip(model_knr.predict(X_Test), a_min=0.0, a_max=None)

average of multiple models

In [0]:
#del model_gbr, model_knr, X_Train, X_Test
pred_train_mix = (pred_train_gbr + np.ravel(pred_train_knr))/2
pred_val_mix = (pred_val_gbr + np.ravel(pred_val_knr))/2

fit and predict

In [0]:
print('GradientBoostingRegressor RMSLE: Train/Validation', rmsle(Y_Train, pred_train_gbr), rmsle(Y_Test, pred_val_gbr))
#del pred_train_gbr, pred_val_gbr
print('KNeighborsRegressor RMSLE: Train/Validation', rmsle(Y_Train, pred_train_knr), rmsle(Y_Test, pred_val_knr))
#del pred_train_knr, pred_val_knr
print('MixedRegressor RMSLE: Train/Validation', rmsle(Y_Train, pred_train_mix), rmsle(Y_Test, pred_val_mix))

GradientBoostingRegressor RMSLE: Train/Validation 0.24939823725547347 0.45681711381164236
KNeighborsRegressor RMSLE: Train/Validation 0.37035238005920523 0.4520408423045235
MixedRegressor RMSLE: Train/Validation 0.30078850296564164 0.44562087373410325


In [0]:
model_gbr.feature_importances_

array([0.01833235, 0.0588238 , 0.00066436, 0.08515765, 0.18128269,
       0.04421542, 0.04291562, 0.00821666, 0.0782245 , 0.48216694])